In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
%matplotlib inline
df = pd.read_csv(r'\\hvcorp1\sharedir\\public\\Data Analytics\\Jedi Order\\Price Optimization\\cleaned_training_data.csv')

In [2]:
df.to_csv(r'\\hvcorp1\sharedir\\public\\Data Analytics\\Jedi Order\\Price Optimization\\cleaned_training_data.csv',index=False)

In [3]:
df.columns.values

array(['actualdate', 'upc', 'mfsfilenumber', 'storenumber',
       'fyweekendingdateint', 'category', 'subcategory', 'parentcompany',
       'parentbrand', 'privatelabel', 'nielsencategory',
       'nielsensubcategory', 'nielsenparentbrand', 'corporatedepartment',
       'organic', 'commodity', 'subcommodity', 'size', 'planogramstatus',
       'regionname', 'storetypedescription', 'new', 'marketdescription',
       'districtname', 'locationtypedescription', 'storeformat',
       'storesize', 'population_level', 'averageage_level',
       'personsperhousehold_level', 'householdincome_level',
       'percapitaincome_level', 'percent_white_level',
       'percent_africanamerican_level', 'percent_hispanic_level',
       'percent_asian_level', 'percent_other_level', 'college_level',
       'population_change_1_yr', 'population_change_10_yr', 'averageage',
       'population', 'avgdiscount', 'promotiontype',
       'featuretypedescription', 'promotionlengthindays',
       'lineitemonpromotio

In [4]:
uniqueUpcList = list(pd.Series(df['upc']))

In [5]:
# dfupc.rename(columns={'7dmovement': 'sevendmovement', '7davgretail': 'sevendavgretail'}, inplace = True)

In [6]:
# create three dummy variables using get_dummies, then exclude the first dummy column
storesize_dummies = pd.get_dummies(df.storesize, prefix='storesize').iloc[:, 1:]

# concatenate the dummy variable columns onto the original DataFrame (axis=0 means rows, axis=1 means columns)
df = pd.concat([df, storesize_dummies], axis=1)
df.shape

(292744, 241)

In [7]:
upcCount = dict(pd.Series(df['upc'].value_counts()))
upcToDrop = []
for key in upcCount:
    if upcCount[key] < 50:
        upcToDrop.append(key)
print(len(upcToDrop))

for item in uniqueUpcList:
    if item in upcToDrop:
        uniqueUpcList.remove(item)



256


In [8]:
# dfupc = df[df['upc'].isin([28200004650])]

In [9]:
argstring = 'sevendmovement ~'
for colname in dfupc.drop(columns=['upc','sevendmovement', 'quantity']): # add additional column names to exclude them as features that are considered by the model
    argstring = argstring + (' ' + colname + ' +')
argstring = argstring[:-2]
argstring

NameError: name 'dfupc' is not defined

In [ ]:
upcModelDict = {}

# run an ols model for each upc with > 50 observations
for uniqueUpc in uniqueUpcList:
    lm = smf.ols(formula=argstring,data= df[df['upc'].isin([uniqueUpc])]).fit()
    tempdict = dict(lm.pvalues)
    finaldict = {}
    for item in tempdict:
        if tempdict[item] < 0.15:
            finaldict[item] = tempdict[item]
    if len(finaldict) > 1:
        upcModelDict[uniqueUpc] = finaldict


In [ ]:
listOfFeatures = []
for key in upcModelDict: # this will create a list of the sets of significant features for each upc model
    listOfFeatures.append(set(upcModelDict[key].keys()))

finalFeaturesFrequency = {} # create an empty dictionary with the purpose of storing the frequency that each feature appeared to be significant
for nextSet in listOfFeatures: # iterate through each set in the list of features
    for key in nextSet:
        try: # add one if the key exists
            finalFeaturesFrequency[key] += 1
        except KeyError: # otherwise create the key and set its value to one
            finalFeaturesFrequency[key] = 1

finalFeaturesProportion = finalFeaturesFrequency.copy() # make a copy of the frequency dict, to replace the value with the proportion

for key in finalFeaturesProportion:
    finalFeaturesProportion[key] = round(finalFeaturesProportion[key]/len(listOfFeatures),3)

In [ ]:
s = [(k, finalFeaturesProportion[k]) for k in sorted(finalFeaturesProportion, key=finalFeaturesProportion.get, reverse=True)]
s

# mean absolute percentage error

In [ ]:
# verfiy model using statsmodels

# create a fitted model with multiple features
lm = smf.ols(formula=argstring, data=dfupc).fit()

# print the coefficients
lm.params

In [ ]:
print('item\tpval')

for item in pvalDict:
    if pvalDict[item] < 0.15:
        print(item,pvalDict[item],sep='\t')
    